# CHATBOTS - Using Natural Language Processing and Tensorflow

In this Jupyter Notebook, We are going to Build a Chatbot that Understands the Context of Sentense and Respond accordingly.
These are the Things that we are going to do in this Project -
1. Transforming the Conversational Intents into Tensorflow model (Neural Network using TFLEARN) using NLP and Save it as Pickle also.
2. Load the Same Pickle and Model to Build the Framework to Process the Responses.
3. At Last, We Show How the Inputs are Processed and Give the Reponses.
-------------------------------------------------------------------------------------------------------

##### TFLEARN  - TFlearn is a modular and transparent deep learning library built on top of Tensorflow. It was designed to provide a higher-level API to TensorFlow in order to facilitate and speed-up experimentations, while remaining fully transparent and compatible with it.  (http://tflearn.org/)
-------------------------------------------------------------------------------------------------------
##### TENSORFLOW - TensorFlow is an end-to-end open source platform for machine learning. It has a comprehensive, flexible ecosystem of tools, libraries and community resources that lets researchers push the state-of-the-art in ML and developers easily build and deploy ML powered applications.


Libraries ------------------------------------------------------------------------------#FreeBirdsCrew

In [ ]:
!pip install tensorflow==2.13.0
!pip install torch==2.3.1 transformers

^C
  Obtaining dependency information for typing-extensions<4.6.0,>=3.6.6 from https://files.pythonhosted.org/packages/31/25/5abcd82372d3d4a3932e1fa8c3dbf9efac10cc7c0d16e78467460571b404/typing_extensions-4.5.0-py3-none-any.whl.metadata
  Using cached typing_extensions-4.5.0-py3-none-any.whl.metadata (8.5 kB)
Using cached typing_extensions-4.5.0-py3-none-any.whl (27 kB)
  Attempting uninstall: typing-extensions
    Found existing installation: typing_extensions 4.12.2
    Uninstalling typing_extensions-4.12.2:
      Successfully uninstalled typing_extensions-4.12.2


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.3.1 requires typing-extensions>=4.8.0, but you have typing-extensions 4.5.0 which is incompatible.


In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from transformers import BertTokenizer, TFBertForSequenceClassification
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.losses import SparseCategoricalCrossentropy
import nltk
from nltk.stem.lancaster import LancasterStemmer
import json
import pickle
import warnings
import random
import torch
from torch.utils.data import DataLoader, TensorDataset, RandomSampler

# Initialize stemmer and download punkt tokenizer
stemmer = LancasterStemmer()
nltk.download('punkt')

warnings.filterwarnings("ignore")

C:\Users\User\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
C:\Users\User\anaconda3\Lib\site-packages\transformers\utils\generic.py:260: UserWarning: torch.utils._pytree._register_pytree_node is deprecated. Please use torch.utils._pytree.register_pytree_node instead.
  torch.utils._pytree._register_pytree_node(
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\User\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [2]:
print("Processing the Intents.....")
with open('intents.json') as json_data:
    intents = json.load(json_data)

Processing the Intents.....


In [3]:
words = []
classes = []
documents = []
ignore_words = ['?']

print("Looping through the Intents to Convert them to words, classes, documents and ignore_words.......")
for intent in intents['intents']:
    for pattern in intent['patterns']:
        # tokenize each word in the sentence
        w = nltk.word_tokenize(pattern)
        # add to our words list
        words.extend(w)
        # add to documents in our corpus
        documents.append((w, intent['tag']))
        # add to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

Looping through the Intents to Convert them to words, classes, documents and ignore_words.......


In [4]:
print("Stemming, Lowering and Removing Duplicates.......")
words = [stemmer.stem(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

# remove duplicates
classes = sorted(list(set(classes)))

print (len(documents), "documents")
print (len(classes), "classes", classes)
print (len(words), "unique stemmed words", words)

Stemming, Lowering and Removing Duplicates.......
271 documents
80 classes ['about', 'afternoon', 'anxious', 'ask', 'casual', 'creation', 'death', 'default', 'depressed', 'done', 'evening', 'fact-1', 'fact-10', 'fact-11', 'fact-12', 'fact-13', 'fact-14', 'fact-15', 'fact-16', 'fact-17', 'fact-18', 'fact-19', 'fact-2', 'fact-20', 'fact-21', 'fact-22', 'fact-23', 'fact-24', 'fact-25', 'fact-26', 'fact-27', 'fact-28', 'fact-29', 'fact-3', 'fact-30', 'fact-31', 'fact-32', 'fact-5', 'fact-6', 'fact-7', 'fact-8', 'fact-9', 'friends', 'goodbye', 'greeting', 'happy', 'hate-me', 'hate-you', 'help', 'jokes', 'learn-mental-health', 'learn-more', 'location', 'meditation', 'mental-health-fact', 'morning', 'name', 'neutral-response', 'night', 'no-approach', 'no-response', 'not-talking', 'pandora-useful', 'problem', 'repeat', 'sad', 'scared', 'skill', 'sleep', 'something-else', 'stressed', 'stupid', 'suicide', 'thanks', 'understand', 'user-advice', 'user-agree', 'user-meditation', 'worthless', 'wrong

In [5]:
print("Creating the Data for our Model.....")
training = []
output = []
print("Creating an List (Empty) for Output.....")
output_empty = [0] * len(classes)

print("Creating Training Set, Bag of Words for our Model....")
for doc in documents:
    # initialize our bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # stem each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    # create our bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])

Creating the Data for our Model.....
Creating an List (Empty) for Output.....
Creating Training Set, Bag of Words for our Model....


In [6]:
print("Shuffling Randomly and Converting into Numpy Array for Faster Processing......")
random.shuffle(training)
training = np.array(training, dtype=object)

print("Creating Train and Test Lists.....")
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Building Neural Network for Out Chatbot to be Contextual....")
print("Resetting graph data....")

Shuffling Randomly and Converting into Numpy Array for Faster Processing......
Creating Train and Test Lists.....
Building Neural Network for Out Chatbot to be Contextual....
Resetting graph data....


In [7]:
print("Building Neural Network for Our Chatbot to be Contextual....")
model = keras.Sequential()
model.add(layers.Input(shape=(len(train_x[0]),)))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(8, activation='relu'))
model.add(layers.Dense(len(train_y[0]), activation='softmax'))

Building Neural Network for Our Chatbot to be Contextual....


In [8]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [9]:
train_x = np.array(train_x)
train_y = np.array(train_y)
print("Training the Model.......")
model.fit(train_x, train_y, epochs=1000, batch_size=8, verbose=1)
print("Saving the Model.......")
model.save('model.keras')


Training the Model.......
Epoch 1/1000
34/34 [==============================] - 0s 1ms/step - loss: 4.3800 - accuracy: 0.0074 
Epoch 2/1000
34/34 [==============================] - 0s 879us/step - loss: 4.3593 - accuracy: 0.0517
Epoch 3/1000
34/34 [==============================] - 0s 788us/step - loss: 4.3392 - accuracy: 0.0517
Epoch 4/1000
34/34 [==============================] - 0s 818us/step - loss: 4.3132 - accuracy: 0.0517
Epoch 5/1000
34/34 [==============================] - 0s 879us/step - loss: 4.2796 - accuracy: 0.0517
Epoch 6/1000
34/34 [==============================] - 0s 788us/step - loss: 4.2356 - accuracy: 0.0554
Epoch 7/1000
34/34 [==============================] - 0s 856us/step - loss: 4.1808 - accuracy: 0.0554
Epoch 8/1000
34/34 [==============================] - 0s 788us/step - loss: 4.1168 - accuracy: 0.0590
Epoch 9/1000
34/34 [==============================] - 0s 758us/step - loss: 4.0487 - accuracy: 0.0627
Epoch 10/1000
34/34 [==============================] - 0s

34/34 [==============================] - 0s 727us/step - loss: 0.4699 - accuracy: 0.9373
Epoch 81/1000
34/34 [==============================] - 0s 848us/step - loss: 0.4530 - accuracy: 0.9446
Epoch 82/1000
34/34 [==============================] - 0s 832us/step - loss: 0.4374 - accuracy: 0.9483
Epoch 83/1000
34/34 [==============================] - 0s 727us/step - loss: 0.4225 - accuracy: 0.9410
Epoch 84/1000
34/34 [==============================] - 0s 1ms/step - loss: 0.4071 - accuracy: 0.9520
Epoch 85/1000
34/34 [==============================] - 0s 818us/step - loss: 0.3933 - accuracy: 0.9557
Epoch 86/1000
34/34 [==============================] - 0s 758us/step - loss: 0.3817 - accuracy: 0.9557
Epoch 87/1000
34/34 [==============================] - 0s 727us/step - loss: 0.3662 - accuracy: 0.9594
Epoch 88/1000
34/34 [==============================] - 0s 849us/step - loss: 0.3539 - accuracy: 0.9557
Epoch 89/1000
34/34 [==============================] - 0s 788us/step - loss: 0.3442 - acc

34/34 [==============================] - 0s 818us/step - loss: 0.0491 - accuracy: 0.9889
Epoch 160/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0478 - accuracy: 0.9926
Epoch 161/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0467 - accuracy: 0.9926
Epoch 162/1000
34/34 [==============================] - 0s 879us/step - loss: 0.0460 - accuracy: 0.9926
Epoch 163/1000
34/34 [==============================] - 0s 953us/step - loss: 0.0447 - accuracy: 0.9926
Epoch 164/1000
34/34 [==============================] - 0s 849us/step - loss: 0.0443 - accuracy: 0.9926
Epoch 165/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0428 - accuracy: 0.9926
Epoch 166/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0423 - accuracy: 0.9889
Epoch 167/1000
34/34 [==============================] - 0s 818us/step - loss: 0.0425 - accuracy: 0.9889
Epoch 168/1000
34/34 [==============================] - 0s 758us/step - loss: 0

34/34 [==============================] - 0s 757us/step - loss: 0.0158 - accuracy: 0.9926
Epoch 238/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0165 - accuracy: 0.9963
Epoch 239/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0147 - accuracy: 0.9963
Epoch 240/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0156 - accuracy: 0.9926
Epoch 241/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0141 - accuracy: 0.9963
Epoch 242/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0158 - accuracy: 0.9963
Epoch 243/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0147 - accuracy: 0.9926
Epoch 244/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0152 - accuracy: 0.9926
Epoch 245/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0139 - accuracy: 0.9926
Epoch 246/1000
34/34 [==============================] - 0s 727us/step - loss: 0

34/34 [==============================] - 0s 727us/step - loss: 0.0082 - accuracy: 0.9963
Epoch 316/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0089 - accuracy: 0.9926
Epoch 317/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0094 - accuracy: 0.9963
Epoch 318/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0078 - accuracy: 0.9926
Epoch 319/1000
34/34 [==============================] - 0s 830us/step - loss: 0.0079 - accuracy: 0.9963
Epoch 320/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0080 - accuracy: 0.9963
Epoch 321/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0089 - accuracy: 0.9926
Epoch 322/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0078 - accuracy: 0.9963
Epoch 323/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0089 - accuracy: 0.9926
Epoch 324/1000
34/34 [==============================] - 0s 758us/step - loss: 0

...

34/34 [==============================] - 0s 727us/step - loss: 0.0073 - accuracy: 0.9926
Epoch 628/1000
34/34 [==============================] - 0s 728us/step - loss: 0.0067 - accuracy: 0.9926
Epoch 629/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0068 - accuracy: 0.9926
Epoch 630/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0056 - accuracy: 0.9963
Epoch 631/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0065 - accuracy: 0.9963
Epoch 632/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0057 - accuracy: 0.9963
Epoch 633/1000
34/34 [==============================] - 0s 700us/step - loss: 0.0066 - accuracy: 0.9963
Epoch 634/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0054 - accuracy: 0.9926
Epoch 635/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0053 - accuracy: 0.9963
Epoch 636/1000
34/34 [==============================] - 0s 727us/step - loss: 0

34/34 [==============================] - 0s 727us/step - loss: 0.0054 - accuracy: 0.9963
Epoch 706/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0070 - accuracy: 0.9963
Epoch 707/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0061 - accuracy: 0.9963
Epoch 708/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0060 - accuracy: 0.9926
Epoch 709/1000
34/34 [==============================] - 0s 732us/step - loss: 0.0066 - accuracy: 0.9963
Epoch 710/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0070 - accuracy: 0.9926
Epoch 711/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0053 - accuracy: 0.9963
Epoch 712/1000
34/34 [==============================] - 0s 732us/step - loss: 0.0063 - accuracy: 0.9926
Epoch 713/1000
34/34 [==============================] - 0s 788us/step - loss: 0.0070 - accuracy: 0.9926
Epoch 714/1000
34/34 [==============================] - 0s 727us/step - loss: 0

34/34 [==============================] - 0s 764us/step - loss: 0.0055 - accuracy: 0.9963
Epoch 784/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0065 - accuracy: 0.9926
Epoch 785/1000
34/34 [==============================] - 0s 762us/step - loss: 0.0062 - accuracy: 0.9926
Epoch 786/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0070 - accuracy: 0.9963
Epoch 787/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0073 - accuracy: 0.9963
Epoch 788/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0056 - accuracy: 0.9963
Epoch 789/1000
34/34 [==============================] - 0s 757us/step - loss: 0.0061 - accuracy: 0.9963
Epoch 790/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0051 - accuracy: 0.9963
Epoch 791/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0066 - accuracy: 0.9926
Epoch 792/1000
34/34 [==============================] - 0s 727us/step - loss: 0

34/34 [==============================] - 0s 727us/step - loss: 0.0088 - accuracy: 0.9926
Epoch 862/1000
34/34 [==============================] - 0s 728us/step - loss: 0.0059 - accuracy: 0.9926
Epoch 863/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0063 - accuracy: 0.9926
Epoch 864/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0066 - accuracy: 0.9926
Epoch 865/1000
34/34 [==============================] - 0s 863us/step - loss: 0.0060 - accuracy: 0.9926
Epoch 866/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0068 - accuracy: 0.9926
Epoch 867/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0065 - accuracy: 0.9926
Epoch 868/1000
34/34 [==============================] - 0s 733us/step - loss: 0.0063 - accuracy: 0.9963
Epoch 869/1000
34/34 [==============================] - 0s 759us/step - loss: 0.0063 - accuracy: 0.9963
Epoch 870/1000
34/34 [==============================] - 0s 697us/step - loss: 0

34/34 [==============================] - 0s 789us/step - loss: 0.0058 - accuracy: 0.9926
Epoch 940/1000
34/34 [==============================] - 0s 735us/step - loss: 0.0068 - accuracy: 0.9926
Epoch 941/1000
34/34 [==============================] - 0s 733us/step - loss: 0.0085 - accuracy: 0.9926
Epoch 942/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0057 - accuracy: 0.9963
Epoch 943/1000
34/34 [==============================] - 0s 727us/step - loss: 0.0055 - accuracy: 0.9926
Epoch 944/1000
34/34 [==============================] - 0s 757us/step - loss: 0.0057 - accuracy: 0.9926
Epoch 945/1000
34/34 [==============================] - 0s 697us/step - loss: 0.0065 - accuracy: 0.9963
Epoch 946/1000
34/34 [==============================] - 0s 758us/step - loss: 0.0064 - accuracy: 0.9926
Epoch 947/1000
34/34 [==============================] - 0s 730us/step - loss: 0.0069 - accuracy: 0.9963
Epoch 948/1000
34/34 [==============================] - 0s 727us/step - loss: 0

In [10]:
print("Pickle is also Saved..........")
pickle.dump( {'words':words, 'classes':classes, 'train_x':train_x, 'train_y':train_y}, open( "training_data", "wb" ) )

Pickle is also Saved..........


In [11]:
print("Loading Pickle.....")
data = pickle.load( open( "training_data", "rb" ) )
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']
    
print("Loading the Model......")
# load our saved model
model = keras.models.load_model('model.keras')

Loading Pickle.....
Loading the Model......


In [12]:
# Initialize BERT tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Prepare data for BERT model
texts = []
labels = []
classes = []
for intent in intents['intents']:
    if intent['tag'] not in classes:
        classes.append(intent['tag'])
    for pattern in intent['patterns']:
        texts.append(pattern)
        labels.append(classes.index(intent['tag']))

In [13]:
def encode_data(texts, labels, tokenizer, max_length=64):
    input_ids = []
    attention_masks = []
    label_ids = []

    for text, label in zip(texts, labels):
        encoded_dict = tokenizer.encode_plus(
            text,
            add_special_tokens=True,
            max_length=max_length,
            padding='max_length',
            return_attention_mask=True,
            return_tensors='tf',
        )
        input_ids.append(encoded_dict['input_ids'])
        attention_masks.append(encoded_dict['attention_mask'])
        label_ids.append(label)

    input_ids = tf.concat(input_ids, axis=0)
    attention_masks = tf.concat(attention_masks, axis=0)
    label_ids = tf.convert_to_tensor(label_ids)

    return input_ids, attention_masks, label_ids

input_ids, attention_masks, label_ids = encode_data(texts, labels, tokenizer)

In [14]:
# Convert to tensors if they are not already
input_ids = tf.convert_to_tensor(input_ids)
attention_masks = tf.convert_to_tensor(attention_masks)
label_ids = tf.convert_to_tensor(label_ids)

# Create a dataset from tensors
dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': input_ids,
    'attention_mask': attention_masks
}, label_ids))

# Create a validation split
validation_size = int(0.1 * len(input_ids))
train_size = len(input_ids) - validation_size

# Split data into training and validation datasets
train_dataset = dataset.take(train_size)
val_dataset = dataset.skip(train_size)

# Define batch size
batch_size = 8

# Prepare the datasets for training
train_dataset = train_dataset.shuffle(1000).batch(batch_size).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(batch_size).prefetch(tf.data.AUTOTUNE)

In [15]:
from tensorflow.keras.callbacks import ReduceLROnPlateau

# Reduce learning rate when a metric has stopped improving
lr_scheduler = ReduceLROnPlateau(monitor='loss', factor=0.1, patience=2, min_lr=1e-7)

# Define the input shape explicitly
input_ids = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name="input_ids")
attention_mask = tf.keras.layers.Input(shape=(64,), dtype=tf.int32, name="attention_mask")

In [16]:
from transformers import TFBertModel

# Use the BERT model
bert_model = TFBertModel.from_pretrained('bert-base-uncased', trainable=True)
sequence_output = bert_model([input_ids, attention_mask])[0]

# Add dropout and a dense layer
dropout = tf.keras.layers.Dropout(0.3)(sequence_output[:, 0, :])  # Use only the [CLS] token output
output = tf.keras.layers.Dense(len(classes), activation='softmax')(dropout)

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFBertModel: ['cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing TFBertModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFBertModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFBertModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFBertModel for predictions w

In [17]:
# Build and compile the model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)
optimizer = Adam(learning_rate=2e-5)
loss = SparseCategoricalCrossentropy(from_logits=False)  # Ensure from_logits is False for softmax activation
model.compile(optimizer=optimizer, loss=loss, metrics=['accuracy'])

# Train BERT model
print("Training BERT model......")
history = model.fit(
    train_dataset,
    validation_data=val_dataset,
    epochs=8,
    callbacks=[lr_scheduler]
)

# Save the BERT model
model.save('bert_model.keras')

Training BERT model......
Epoch 1/8
31/31 [==============================] - 58s 1s/step - loss: 4.5379 - accuracy: 0.0246 - val_loss: 4.6292 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 2/8
31/31 [==============================] - 40s 1s/step - loss: 3.6162 - accuracy: 0.2008 - val_loss: 5.6750 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 3/8
31/31 [==============================] - 39s 1s/step - loss: 2.4573 - accuracy: 0.4918 - val_loss: 5.7561 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 4/8
31/31 [==============================] - 39s 1s/step - loss: 1.6046 - accuracy: 0.7213 - val_loss: 5.9022 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 5/8
31/31 [==============================] - 39s 1s/step - loss: 0.9144 - accuracy: 0.8934 - val_loss: 6.0611 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 6/8
31/31 [==============================] - 39s 1s/step - loss: 0.6126 - accuracy: 0.9180 - val_loss: 6.1705 - val_accuracy: 0.0000e+00 - lr: 2.0000e-05
Epoch 7/8
31/3

In [29]:
def clean_up_sentence(sentence):
    # Tokenize the sentence
    sentence_words = nltk.word_tokenize(sentence)
    # Stemming means to find the root of the word.
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

# Return the Array of Bag of Words: True or False and 0 or 1 for each word of bag that exists in the Sentence
def bow(sentence, words, show_details=False):
    sentence_words = clean_up_sentence(sentence)
    bag = [0]*len(words)
    for s in sentence_words:
        for i,w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print ("found in bag: %s" % w)
    return np.array(bag[:64])

ERROR_THRESHOLD = 0.25
print("ERROR_THRESHOLD = 0.25")

ERROR_THRESHOLD = 0.25


In [30]:
def classify_with_traditional_model(sentence):
    input_data = np.array([bow(sentence, words)])
    attention_mask = np.ones(input_data.shape)  # Create a dummy attention mask
    results = model.predict([input_data, attention_mask])[0]
    results = [[i, r] for i, r in enumerate(results) if r > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [(classes[r[0]], r[1]) for r in results]
    return return_list

def classify_with_bert_model(sentence):
    inputs = tokenizer.encode_plus(
        sentence,
        add_special_tokens=True,
        max_length=64,
        padding='max_length',
        return_attention_mask=True,
        return_tensors='tf',
    )
    input_ids = inputs['input_ids']
    attention_mask = inputs['attention_mask']

    # Make sure to pass the inputs as a dictionary
    logits = model.predict({'input_ids': input_ids, 'attention_mask': attention_mask})[0]
    
    # Make sure to use the softmax function on the correct dimension
    probs = tf.nn.softmax(logits, axis=-1).numpy().flatten()
    results = [[i, prob] for i, prob in enumerate(probs) if prob > ERROR_THRESHOLD]
    results.sort(key=lambda x: x[1], reverse=True)
    return_list = [(classes[r[0]], r[1]) for r in results]
    return return_list

In [35]:
def classify(sentence):
    traditional_results = classify_with_traditional_model(sentence)
    bert_results = classify_with_bert_model(sentence)

    # Remove duplicates and combine results
    combined_results = {tag: score for tag, score in traditional_results + bert_results}
    sorted_results = sorted(combined_results.items(), key=lambda x: x[1], reverse=True)

    return sorted_results

def getIntent(intent):
    intents = json.loads(open('intents.json').read())
    for i in intents['intents']:
        if i['tag'] == intent:
            return random.choice(i['responses'])
    return 'I didnt understand'

def response(sentence, userID='123', show_details=False):
    results = classify(sentence)
    if results:
        while results:
            for i in results:
                if i[1] > ERROR_THRESHOLD:
                    if show_details: print('Intent : ', i[0], 'with a probability score : ', i[1])
                    return getIntent(i[0])
            results.pop(0)
    return 'I didnt understand'

In [ ]:
while True:
    input_data = input("You: ")
    if input_data.lower() == 'quit':
        break
    print(response(input_data))

You: Hi
1/1 [==============================] - 0s 82ms/step
I didnt understand
You: Hello there
1/1 [==============================] - 0s 83ms/step
I didnt understand
